In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
import torch.optim as optim

###############################
# 1. Build the Heterograph    #
###############################

# Lớp kết nối đến cơ sở dữ liệu Neo4j
class Neo4jConnector:
    def __init__(self, uri, user, password):
        # Tạo driver kết nối đến Neo4j với thông tin xác thực
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        print("Connected to Neo4j successfully!")

    def close(self):
        # Đóng kết nối
        self.driver.close()

    def run_query(self, query):
        # Chạy câu truy vấn và trả về kết quả dưới dạng danh sách
        with self.driver.session() as session:
            return list(session.run(query))


In [ ]:
# Kết nối đến cơ sở dữ liệu Neo4j
uri = "bolt://localhost:7687"
user = "neo4j"
password = "12345678"
connector = Neo4jConnector(uri, user, password)

# Truy vấn kết hợp để lấy dữ liệu từ Neo4j
combined_query = """
MATCH (n1)-[r]->(n2)
RETURN id(n1) AS source, id(n2) AS target, type(r) AS relation_type,
       id(n1) AS node1_id, labels(n1) AS node1_labels, n1.property1 AS node1_feature1, n1.property2 AS node1_feature2,
       id(n2) AS node2_id, labels(n2) AS node2_labels, n2.property1 AS node2_feature1, n2.property2 AS node2_feature2
"""
results = connector.run_query(combined_query)
connector.close()  # Đóng kết nối sau khi truy vấn

In [ ]:
# Xử lý dữ liệu để tách thông tin của node và edge
node_features = {}    # Lưu các đặc trưng của node, key: node id, value: dict các feature
node_labels   = {}    # Lưu nhãn của node, key: node id, value: danh sách nhãn
edge_list     = []    # Danh sách các cạnh dưới dạng (source, target, relation_type)

# Duyệt qua từng bản ghi trả về từ Neo4j
for record in results:
    # Xử lý cho cả hai node từ mỗi bản ghi
    for node_id, labels_list, feat1, feat2 in [
        (record["node1_id"], record["node1_labels"], record["node1_feature1"], record["node1_feature2"]),
        (record["node2_id"], record["node2_labels"], record["node2_feature1"], record["node2_feature2"])
    ]:
        if node_id not in node_features:
            # Nếu một node chưa có trong dictionary, thêm vào với các giá trị feature. Nếu không có giá trị, mặc định là 0.
            node_features[node_id] = {
                "feature1": feat1 if feat1 is not None else 0,
                "feature2": feat2 if feat2 is not None else 0
            }
            node_labels[node_id] = labels_list

    # Thêm thông tin cạnh vào danh sách edge_list
    edge_list.append((record["source"], record["target"], record["relation_type"]))


In [ ]:
# Phân loại các node theo nhãn của chúng
user_nodes   = [nid for nid in node_features if "User" in node_labels[nid]]
card_nodes   = [nid for nid in node_features if "Card" in node_labels[nid]]
ip_nodes     = [nid for nid in node_features if "IP" in node_labels[nid]]
device_nodes = [nid for nid in node_features if "Device" in node_labels[nid]]

print(f"Number of 'User' nodes from Neo4j: {len(user_nodes)}")


In [ ]:
# Tạo mapping để ánh xạ id gốc sang chỉ số mới cho mỗi loại node
user_mapping   = {old_id: new_idx for new_idx, old_id in enumerate(user_nodes)}
card_mapping   = {old_id: new_idx for new_idx, old_id in enumerate(card_nodes)}
ip_mapping     = {old_id: new_idx for new_idx, old_id in enumerate(ip_nodes)}
device_mapping = {old_id: new_idx for new_idx, old_id in enumerate(device_nodes)}


In [ ]:
# Chuẩn bị dữ liệu cạnh cho heterograph.
hetero_graph_data = {
    ('User', 'HAS_CC', 'Card'): [],
    ('User', 'HAS_IP', 'IP'): [],
    ('User', 'P2P', 'User'): [],
    ('User', 'P2P_WITH_SHARED_CARD', 'User'): [],
    ('User', 'REFERRED', 'User'): [],
    ('User', 'SHARED_IDS', 'User'): [],
    ('User', 'USED', 'Device'): []
}

# Ánh xạ các quan hệ đến kiểu node đích và mapping tương ứng
relation_to_target = {
    'HAS_CC': ('Card', card_mapping),
    'HAS_IP': ('IP', ip_mapping),
    'P2P': ('User', user_mapping),
    'P2P_WITH_SHARED_CARD': ('User', user_mapping),
    'REFERRED': ('User', user_mapping),
    'SHARED_IDS': ('User', user_mapping),
    'USED': ('Device', device_mapping)
}


In [ ]:
# Duyệt qua danh sách cạnh và thêm cạnh vào hetero_graph_data nếu các node đã tồn tại trong mapping
for src, tgt, rel in edge_list:
    if src not in user_mapping:
        continue
    if rel not in relation_to_target:
        continue
    target_type, tgt_mapping = relation_to_target[rel]
    if target_type == 'User':
        if tgt not in user_mapping:
            continue
        hetero_graph_data[('User', rel, 'User')].append((user_mapping[src], user_mapping[tgt]))
    elif target_type == 'Card':
        if tgt not in card_mapping:
            continue
        hetero_graph_data[('User', rel, 'Card')].append((user_mapping[src], card_mapping[tgt]))
    elif target_type == 'IP':
        if tgt not in ip_mapping:
            continue
        hetero_graph_data[('User', rel, 'IP')].append((user_mapping[src], ip_mapping[tgt]))
    elif target_type == 'Device':
        if tgt not in device_mapping:
            continue
        hetero_graph_data[('User', rel, 'Device')].append((user_mapping[src], device_mapping[tgt]))


In [ ]:
# Xác định số lượng node của mỗi loại
num_nodes_dict = {
    'User': len(user_nodes),
    'Card': len(card_nodes),
    'IP': len(ip_nodes),
    'Device': len(device_nodes)
}


In [ ]:
# Tạo heterograph sử dụng DGL
hetero_graph = dgl.heterograph(hetero_graph_data, num_nodes_dict=num_nodes_dict)
print(f"Number of 'User' nodes in heterograph: {hetero_graph.num_nodes('User')}")
print(f"Number of 'User' nodes in heterograph: {hetero_graph.num_nodes('Card')}")
print(f"Number of 'User' nodes in heterograph: {hetero_graph.num_nodes('IP')}")
print(f"Number of 'User' nodes in heterograph: {hetero_graph.num_nodes('Device')}")

In [ ]:
# Xây dựng ma trận đặc trưng cho các node theo thứ tự đã được re-index
ordered_user_features = np.array([
    [node_features[old_id]["feature1"], node_features[old_id]["feature2"]]
    for old_id in user_nodes
])
hetero_graph.nodes['User'].data['features'] = torch.tensor(ordered_user_features, dtype=torch.float32)

ordered_card_features = np.array([
    [node_features[old_id]["feature1"], node_features[old_id]["feature2"]]
    for old_id in card_nodes
])
hetero_graph.nodes['Card'].data['features'] = torch.tensor(ordered_card_features, dtype=torch.float32)

ordered_ip_features = np.array([
    [node_features[old_id]["feature1"], node_features[old_id]["feature2"]]
    for old_id in ip_nodes
])
hetero_graph.nodes['IP'].data['features'] = torch.tensor(ordered_ip_features, dtype=torch.float32)

ordered_device_features = np.array([
    [node_features[old_id]["feature1"], node_features[old_id]["feature2"]]
    for old_id in device_nodes
])
hetero_graph.nodes['Device'].data['features'] = torch.tensor(ordered_device_features, dtype=torch.float32)


In [ ]:
# Tạo tensor nhãn cho node 'User'
# Nếu nhãn của node chứa 'FlaggedUser' thì gán nhãn 1 (gian lận), ngược lại gán 0.
user_labels = []
for old_id in user_nodes:
    if "FlaggedUser" in node_labels[old_id]:
        user_labels.append(1)
    else:
        user_labels.append(0)
labels_tensor = torch.tensor(user_labels, dtype=torch.long)


In [ ]:
# Đếm số lượng nhãn gian lận (1) và không gian lận (0)
num_fraud = torch.sum(labels_tensor == 1).item()
num_non_fraud = torch.sum(labels_tensor == 0).item()

# In kết quả
print(f"Số lượng nút gian lận (1): {num_fraud}")
print(f"Số lượng nút không gian lận (0): {num_non_fraud}")

In [ ]:
#####################################
# 2. Define and Train the HeteroGNN #
#####################################

# Định nghĩa mô hình HeteroGNN sử dụng PyTorch
class HeteroGNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(HeteroGNN, self).__init__()
        # Định nghĩa các lớp GraphConv cho mỗi loại quan hệ trong heterograph
        self.conv1 = dglnn.HeteroGraphConv({
            'HAS_CC': dglnn.GraphConv(input_size, hidden_size),
            'HAS_IP': dglnn.GraphConv(input_size, hidden_size),
            'P2P': dglnn.GraphConv(input_size, hidden_size),
            'P2P_WITH_SHARED_CARD': dglnn.GraphConv(input_size, hidden_size),
            'REFERRED': dglnn.GraphConv(input_size, hidden_size),
            'SHARED_IDS': dglnn.GraphConv(input_size, hidden_size),
            'USED': dglnn.GraphConv(input_size, hidden_size)
        }, aggregate='mean')  # Sử dụng phép trung bình để tổng hợp các đặc trưng từ các quan hệ khác nhau
        self.fc = nn.Linear(hidden_size, output_size)  # Lớp fully-connected chuyển từ không gian ẩn sang đầu ra

    def forward(self, g, features):
        # Tính toán các tầng convolution cho mỗi loại quan hệ
        h = self.conv1(g, features)
        # Áp dụng hàm kích hoạt ReLU cho các đặc trưng thu được (dạng dict với key là loại node)
        h = {ntype: F.relu(feat) for ntype, feat in h.items()}
        # Lấy đặc trưng của node 'User' để dự đoán gian lận
        h_user = h['User']
        return self.fc(h_user)  # Trả về kết quả dự đoán qua lớp fully-connected


In [ ]:
# Lấy kích thước đặc trưng đầu vào từ node 'User'
input_size = hetero_graph.nodes['User'].data['features'].shape[1]
# Khởi tạo mô hình với hidden_size = 64 và output_size = 2 (gian lận hoặc không gian lận)
model = HeteroGNN(input_size=input_size, hidden_size=64, output_size=2)
optimizer = optim.Adam(model.parameters(), lr=1e-3)  # Sử dụng optimizer Adam
criterion = nn.CrossEntropyLoss()  # Hàm mất mát CrossEntropy cho bài toán phân loại


In [ ]:
# Tạo dictionary chứa các đặc trưng của tất cả các loại node
features = {
    'User': hetero_graph.nodes['User'].data['features'],
    'Card': hetero_graph.nodes['Card'].data['features'],
    'IP': hetero_graph.nodes['IP'].data['features'],
    'Device': hetero_graph.nodes['Device'].data['features']
}


In [ ]:
# Vòng lặp huấn luyện mô hình trong 20 epoch
for epoch in range(20):
    model.train()              # Chuyển mô hình sang chế độ training
    optimizer.zero_grad()      # Đặt lại gradient về 0
    predictions = model(hetero_graph, features)  # Tính toán dự đoán từ mô hình
    loss = criterion(predictions, labels_tensor)  # Tính toán hàm mất mát so với nhãn thật
    loss.backward()            # Lan truyền ngược
    optimizer.step()           # Cập nhật trọng số mô hình
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")


In [ ]:
# Đánh giá mô hình sau khi huấn luyện
model.eval()  # Chuyển sang chế độ đánh giá
with torch.no_grad():  # Không tính toán gradient trong quá trình đánh giá
    predictions = model(hetero_graph, features)
    predicted_labels = torch.argmax(predictions, dim=1)  # Lấy nhãn có xác suất cao nhất
    # Đếm số lượng người dùng được dự đoán là gian lận và không gian lận
    num_fraud = (predicted_labels == 1).sum().item()
    num_non_fraud = (predicted_labels == 0).sum().item()
    print("User Fraud Predictions:", predicted_labels.numpy())
    print(f"Total Fraud Users: {num_fraud}")
    print(f"Total Non-Fraud Users: {num_non_fraud}")
    user_node_ids = hetero_graph.nodes('User')

    # Create boolean masks for fraud and non-fraud
    fraud_mask = predicted_labels == 1
    nonfraud_mask = predicted_labels == 0

    # Filter the user node IDs based on the masks.
    fraud_nodes = user_node_ids[fraud_mask]
    nonfraud_nodes = user_node_ids[nonfraud_mask]

    print("Fraud user nodes in hetero graph (indices):", fraud_nodes)
    print("Total fraud user nodes:", fraud_nodes.shape[0])
    print("Non-fraud user nodes in hetero graph (indices):", nonfraud_nodes)
    print("Total non-fraud user nodes:", nonfraud_nodes.shape[0])
